In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from torch.optim import Adam
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

In [2]:
def activation(name):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['lrelu', 'LReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus(beta=4)
    elif name in ['celu', 'CeLU']:
        return nn.CELU()
    elif name in ['elu']:
        return nn.ELU()
    elif name in ['mish']:
        return nn.Mish()
    else:
        raise ValueError('Unknown activation function')




################################################################
#  1d fourier layer
################################################################
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        self.regularization_exp = 1
        self.regularization_param = 0.0000001

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.complex128))


    # Complex multiplication
    def compl_mul1d(self, input, weights):
        #print("einsum: input, weights", input.shape, weights.shape)
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)

        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # x.shape == [batch_size, in_channels, number of grid points]
        # hint: use torch.fft library torch.fft.rfft
        # use DFT to approximate the fourier transform

        # Compute Fourier coefficients
        #print("x before fft", x.shape)
        x_ft = torch.fft.rfft(x)
        #print("x after fft", x.shape)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1) // 2 + 1, device=x.device, dtype=torch.complex128)
        #print("outft shape", out_ft.shape)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :,:self.modes1], self.weights1)

        # Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

    def regularization(self):
        reg_loss = 0
        for name, param in self.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(param, self.regularization_exp)
        return self.regularization_param * reg_loss



####################################################################


class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()
        self.modes1 = modes
        self.width = width
        self.padding = 1  # pad the domain if input is non-periodic
        self.linear_p = nn.Linear(3, self.width, dtype=torch.float64)

        self.spect1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.lin0 = nn.Conv1d(self.width, self.width, 1, dtype = torch.float64)
        self.lin1 = nn.Conv1d(self.width, self.width, 1, dtype = torch.float64)
        self.lin2 = nn.Conv1d(self.width, self.width, 1, dtype = torch.float64)

        self.linear_q = nn.Linear(self.width, 32, dtype = torch.float64)
        self.output_layer = nn.Linear(32, 2, dtype = torch.float64)

        self.activation = torch.nn.Tanh()

    def fourier_layer(self, x, spectral_layer, conv_layer):
        return self.activation(spectral_layer(x) + conv_layer(x))

    def linear_layer(self, x, linear_transformation):
        return self.activation(linear_transformation(x))

    def forward(self, x):
        # grid = self.get_grid(x.shape, x.device)
        # x = torch.cat((x, grid), dim=-1)
        #print("x before linear_ü",x.shape)
        x = self.linear_p(x)
        #print("x after linear_p",x.shape)
        x = x.permute(0, 2, 1)
        #print("x after permute",x.shape)

        #x = F.pad(x, [0, self.padding])  # pad the domain if input is non-periodic

        x = self.fourier_layer(x, self.spect1, self.lin0)
        x = self.fourier_layer(x, self.spect2, self.lin1)
        x = self.fourier_layer(x, self.spect3, self.lin2)

        #x = x[..., :-self.padding]  # pad the domain if input is non-periodic

        x = x.permute(0, 2, 1)
        #print("x after permute",x.shape)

        x = self.linear_layer(x, self.linear_q)
        x = self.output_layer(x)
        return x

    def compute_loss(self, pred, train):
        l = torch.nn.MSELoss()
        loss = l(pred, train) + self.spect1.regularization() + self.spect2.regularization() + self.spect3.regularization()
        #loss = tuple(loss)
        return loss




In [3]:
torch.manual_seed(42)
np.random.seed(42)
import pandas as pd
n_train = 106

df = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TrainingData.txt')
data_read= df.iloc[:,0:3].values

data_read = torch.tensor(data_read).to(torch.float64)
print(data_read.shape)

Time_Column = torch.clone(data_read[:, 0])
Tf_Column = torch.clone(data_read[:, 1])
Ts_Column = torch.clone(data_read[:, 2])


data_read[:, 0] = Tf_Column
data_read[:, 1] = Ts_Column
data_read[:, 2] = Time_Column


max_Tf = torch.max(data_read[:,0])
max_Ts = torch.max(data_read[:,1])


data_read[:,0] /= max_Tf
data_read[:,1] /= max_Ts
data_read[:,2] /= 602168.58


data_read = data_read.unsqueeze(0).to(device)


data_altered = torch.tensor((), dtype=torch.float64).to(device)
WINDOW_SIZE = 35
NUM_MEASUREMENTS = data_read.shape[1]

n_intervals = NUM_MEASUREMENTS - (WINDOW_SIZE - 1)

for i in range(0, n_intervals):
    data_altered = torch.cat(
        (data_altered, data_read[:, i : 35 + i]), dim=0)
print(data_altered.shape)

x_data = data_altered[0:141,:,0:3]
y_data = data_altered[35:176,:,0:2]


print(x_data.shape,y_data.shape)
#print("read value is:", data_read[:,173,0])
#print("our value is: ", x_data[140, 34, 0])
input_function_train = x_data#[:n_train, :]
output_function_train = y_data#[:n_train, :]
#input_function_test = x_data[n_train:, :]
#output_function_test = y_data[n_train:, :]


batch_size = 141

training_set = DataLoader(TensorDataset(input_function_train, output_function_train), batch_size=batch_size, shuffle=True)
#testing_set = DataLoader(TensorDataset(input_function_test, output_function_test), batch_size=batch_size, shuffle=False)


torch.Size([210, 3])
torch.Size([176, 35, 3])
torch.Size([141, 35, 3]) torch.Size([141, 35, 2])


In [ ]:
learning_rate = 0.001

# epochs = 250
epochs = 4000
step_size = 100
gamma = 0.75

modes = 18
width = 256

# model

fno = FNO1d(modes, width).to(device)

optimizer = Adam(fno.parameters(), lr=learning_rate, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


l = torch.nn.MSELoss()
freq_print = 1
for epoch in range(epochs):
    train_mse = 0.0
    for step, (input_batch, output_batch) in enumerate(training_set):
        optimizer.zero_grad()
        output_pred_batch = fno(input_batch)
        output_pred_batch = fno(input_batch).squeeze(0)
        output_batch = output_batch.squeeze(0)
        loss_f = torch.log10(l(output_pred_batch, output_batch))
        loss_f.backward()
        optimizer.step()
        train_mse += loss_f.item()
    train_mse /= len(training_set)

    scheduler.step()
    """
    with torch.no_grad():
        fno.eval()
        test_relative_l2 = 0.0
        for step, (input_batch, output_batch) in enumerate(testing_set):
            output_pred_batch = fno(input_batch).squeeze(2)
            loss_f = torch.log10(l(output_pred_batch, output_batch))
            test_relative_l2 += loss_f.item()
        test_relative_l2 /= len(testing_set)
     """

    if epoch % freq_print == 0: print("######### Epoch:", epoch, " ######### Train Loss:", train_mse,)#" ######### Relative L2 Test Norm:", test_relative_l2)



######### Epoch: 0  ######### Train Loss: -0.1324861098802973
######### Epoch: 1  ######### Train Loss: -1.7336442430227017
######### Epoch: 2  ######### Train Loss: -0.3276130919438893
######### Epoch: 3  ######### Train Loss: -0.06188090754908181
######### Epoch: 4  ######### Train Loss: -0.03466868437544145
######### Epoch: 5  ######### Train Loss: -0.11501956501918939
######### Epoch: 6  ######### Train Loss: -0.31195111037198386
######### Epoch: 7  ######### Train Loss: -0.7507355289244905
######### Epoch: 8  ######### Train Loss: -2.4340939767256002
######### Epoch: 9  ######### Train Loss: -0.9398601758217736
######### Epoch: 10  ######### Train Loss: -0.5435266871377394
######### Epoch: 11  ######### Train Loss: -0.4107392125885409
######### Epoch: 12  ######### Train Loss: -0.3777779195046619
######### Epoch: 13  ######### Train Loss: -0.3991698449037402
######### Epoch: 14  ######### Train Loss: -0.45656341376472637
######### Epoch: 15  ######### Train Loss: -0.54173592845462

######### Epoch: 132  ######### Train Loss: -3.3721004113795043
######### Epoch: 133  ######### Train Loss: -3.233150608130064
######### Epoch: 134  ######### Train Loss: -3.170395875358696
######### Epoch: 135  ######### Train Loss: -3.152727471177277
######### Epoch: 136  ######### Train Loss: -3.1823505034438035
######### Epoch: 137  ######### Train Loss: -3.2758544822030133
######### Epoch: 138  ######### Train Loss: -3.464768514255291
######### Epoch: 139  ######### Train Loss: -3.8073826959409236
######### Epoch: 140  ######### Train Loss: -4.19525231827208
######### Epoch: 141  ######### Train Loss: -4.282929430631962
######### Epoch: 142  ######### Train Loss: -4.023697808974123
######### Epoch: 143  ######### Train Loss: -3.960423522579649
######### Epoch: 144  ######### Train Loss: -4.044586588142116
######### Epoch: 145  ######### Train Loss: -4.214382612561822
######### Epoch: 146  ######### Train Loss: -4.476892555134925
######### Epoch: 147  ######### Train Loss: -4.46971

######### Epoch: 262  ######### Train Loss: -4.425990951197386
######### Epoch: 263  ######### Train Loss: -4.600011618258762
######### Epoch: 264  ######### Train Loss: -4.727276661126264
######### Epoch: 265  ######### Train Loss: -4.7914035583226555
######### Epoch: 266  ######### Train Loss: -4.793352491049434
######### Epoch: 267  ######### Train Loss: -4.591737014524939
######### Epoch: 268  ######### Train Loss: -4.676053372844677
######### Epoch: 269  ######### Train Loss: -4.79689383067932
######### Epoch: 270  ######### Train Loss: -4.971903806254589
######### Epoch: 271  ######### Train Loss: -4.941514735604045
######### Epoch: 272  ######### Train Loss: -4.661096341711019
######### Epoch: 273  ######### Train Loss: -5.276609967928943
######### Epoch: 274  ######### Train Loss: -4.0977607036605
######### Epoch: 275  ######### Train Loss: -3.7648948192932212
######### Epoch: 276  ######### Train Loss: -3.719590830190854
######### Epoch: 277  ######### Train Loss: -3.848103552

######### Epoch: 392  ######### Train Loss: -3.48083907804641
######### Epoch: 393  ######### Train Loss: -3.5401052987331525
######### Epoch: 394  ######### Train Loss: -3.642814145372925
######### Epoch: 395  ######### Train Loss: -3.7974274453057215
######### Epoch: 396  ######### Train Loss: -4.017030582254886
######### Epoch: 397  ######### Train Loss: -4.290748622157637
######### Epoch: 398  ######### Train Loss: -4.470052816952202
######### Epoch: 399  ######### Train Loss: -4.466280548019511
######### Epoch: 400  ######### Train Loss: -4.413014189898994
######### Epoch: 401  ######### Train Loss: -4.399832804078619
######### Epoch: 402  ######### Train Loss: -4.472256690151084
######### Epoch: 403  ######### Train Loss: -4.706042376557632
######### Epoch: 404  ######### Train Loss: -5.159241510369549
######### Epoch: 405  ######### Train Loss: -4.755986281854339
######### Epoch: 406  ######### Train Loss: -4.651658506753807
######### Epoch: 407  ######### Train Loss: -4.9540900

######### Epoch: 524  ######### Train Loss: -4.6789793085656655
######### Epoch: 525  ######### Train Loss: -4.751608394617475
######### Epoch: 526  ######### Train Loss: -5.243485059512211
######### Epoch: 527  ######### Train Loss: -5.04114640363794
######### Epoch: 528  ######### Train Loss: -4.747415800031484
######### Epoch: 529  ######### Train Loss: -4.813434614225443
######### Epoch: 530  ######### Train Loss: -5.142478659002471
######### Epoch: 531  ######### Train Loss: -5.252656089119221
######### Epoch: 532  ######### Train Loss: -5.286051316008514
######### Epoch: 533  ######### Train Loss: -5.102622808465208
######### Epoch: 534  ######### Train Loss: -5.263815764335168
######### Epoch: 535  ######### Train Loss: -5.44730610189624
######### Epoch: 536  ######### Train Loss: -5.5329787887708095
######### Epoch: 537  ######### Train Loss: -5.169406347814934
######### Epoch: 538  ######### Train Loss: -4.977569707399635
######### Epoch: 539  ######### Train Loss: -5.29376283

######### Epoch: 654  ######### Train Loss: -4.08952342554193
######### Epoch: 655  ######### Train Loss: -4.178214690224338
######### Epoch: 656  ######### Train Loss: -4.338391180342494
######### Epoch: 657  ######### Train Loss: -4.616430242830481
######### Epoch: 658  ######### Train Loss: -5.119150415432334
######### Epoch: 659  ######### Train Loss: -5.151710199562376
######### Epoch: 660  ######### Train Loss: -4.832560649051699
######### Epoch: 661  ######### Train Loss: -4.7904846915481185
######### Epoch: 662  ######### Train Loss: -4.99726183792661
######### Epoch: 663  ######### Train Loss: -5.514932307107868
######### Epoch: 664  ######### Train Loss: -5.102928776783854
######### Epoch: 665  ######### Train Loss: -4.867251973888811
######### Epoch: 666  ######### Train Loss: -4.956239695727897
######### Epoch: 667  ######### Train Loss: -5.46374029485685
######### Epoch: 668  ######### Train Loss: -5.227582970914029
######### Epoch: 669  ######### Train Loss: -4.8889823575

######### Epoch: 786  ######### Train Loss: -5.407598152236391
######### Epoch: 787  ######### Train Loss: -6.06587079116066
######### Epoch: 788  ######### Train Loss: -5.153783045072465
######### Epoch: 789  ######### Train Loss: -4.679497117807573
######### Epoch: 790  ######### Train Loss: -4.4841622550139
######### Epoch: 791  ######### Train Loss: -4.40123057637067
######### Epoch: 792  ######### Train Loss: -4.38700049712042
######### Epoch: 793  ######### Train Loss: -4.427934526238812
######### Epoch: 794  ######### Train Loss: -4.524728061608639
######### Epoch: 795  ######### Train Loss: -4.690789926102016
######### Epoch: 796  ######### Train Loss: -4.955009866068098
######### Epoch: 797  ######### Train Loss: -5.316319829635918
######### Epoch: 798  ######### Train Loss: -5.458135066369055
######### Epoch: 799  ######### Train Loss: -5.404305456747854
######### Epoch: 800  ######### Train Loss: -5.316813349778291
######### Epoch: 801  ######### Train Loss: -5.3020712745728

######### Epoch: 918  ######### Train Loss: -6.000002779638428
######### Epoch: 919  ######### Train Loss: -6.31096961736612
######### Epoch: 920  ######### Train Loss: -5.679088316864967
######### Epoch: 921  ######### Train Loss: -5.322101023693806
######### Epoch: 922  ######### Train Loss: -5.2834347166882525
######### Epoch: 923  ######### Train Loss: -5.469996776504093
######### Epoch: 924  ######### Train Loss: -6.041208892727939
######### Epoch: 925  ######### Train Loss: -5.733547078321426
######### Epoch: 926  ######### Train Loss: -5.407192492135444
######### Epoch: 927  ######### Train Loss: -5.412844680011158
######### Epoch: 928  ######### Train Loss: -5.706855894971564
######### Epoch: 929  ######### Train Loss: -6.199803702723116
######### Epoch: 930  ######### Train Loss: -5.906782757706313
######### Epoch: 931  ######### Train Loss: -5.920259170695008
######### Epoch: 932  ######### Train Loss: -6.039622161837793
######### Epoch: 933  ######### Train Loss: -6.30404803

In [ ]:
test_read = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TestingData.txt')
test_read = test_read.iloc[:,0:1].values
test_read = torch.tensor(test_read).to(torch.float64).squeeze(1).to(device)
test_read /= 602168.58

output_function_test_0 = fno(input_function_train[0,:,:].unsqueeze(0))
output_function_test_1 = fno(input_function_train[35,:,:].unsqueeze(0))
output_function_test_2 = fno(input_function_train[70,:,:].unsqueeze(0))
output_function_test_3 = fno(input_function_train[105,:,:].unsqueeze(0))
output_function_test_4 = fno(input_function_train[140,:,:].unsqueeze(0))
#output_function_test_4 = fno(data_read[:,140:175].to(device))


next_step = torch.cat((output_function_train[140,:,:].unsqueeze(0), data_read[:,175:210, 2].unsqueeze(2)), dim=2)
print(next_step.shape)
output_function_test_5 = fno(next_step)

interval = torch.linspace(520392.6,693856.8, 70).to(device)
interval /=  602168.58
next_step = torch.cat((output_function_test_5, interval[:35].unsqueeze(0).unsqueeze(2)), dim=2)
output_function_test_6 = fno(next_step)

input_function_pred = torch.cat((input_function_train[35,:,2], input_function_train[70,:,2], input_function_train[105,:,2], data_read[:,140:210,2].squeeze(0),interval), dim=0) #data_read[:,175:210,2].squeeze(0).to(device), test_read), dim=0)
output_function_pred = torch.cat((output_function_test_0, output_function_test_1, output_function_test_2, output_function_test_3, output_function_test_4, output_function_test_5, output_function_test_6), dim=1)


input_function_plot = torch.cat((input_function_train[0,:,2], input_function_train[35,:,2], input_function_train[70,:,2], input_function_train[105,:,2], data_read[:,140:210,2].squeeze(0).to(device)), dim=0)
output_function_plot = torch.cat((input_function_train[0,:,0], input_function_train[35,:,0], input_function_train[70,:,0], input_function_train[105,:,0], data_read[:,140:210,0].squeeze(0).to(device)), dim=0)

diff = abs(output_function_pred[:,:175, 0] - output_function_plot[35:]).squeeze(0)

plt.figure(dpi=1500)
plt.grid(True, which="both", ls=":")

plt.plot(input_function_plot.cpu().detach(), output_function_plot.cpu().detach(), label="True Solution", c="C0", lw=2)
plt.plot(input_function_pred.squeeze(0).cpu().detach(), output_function_pred[:,:245,0].squeeze(0).cpu().detach(), label="Approximate Solution", c="C1", lw=1)

#plt.plot(input_function_plot[35:].cpu().detach(), diff.cpu().detach(), label="True Solution", c="C0", lw=2)

plt.legend()
plt.show()


In [ ]:
test_read = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TestingData.txt')
test_read = test_read.iloc[:,0:1].values
test_read = torch.tensor(test_read).to(torch.float64).squeeze(1).to(device)
test_read /= 602168.58

saved_pred = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\tensor_data.txt')
saved_pred = saved_pred.iloc[:,1:3].values
saved_pred = torch.tensor(saved_pred).to(torch.float64).unsqueeze(0).to(device)


output_function_test_0 = fno(input_function_train[0,:,:].unsqueeze(0))
output_function_test_1 = fno(input_function_train[1,:,:].unsqueeze(0))
output_function_test_2 = fno(input_function_train[2,:,:].unsqueeze(0))
output_function_test_3 = fno(input_function_train[3,:,:].unsqueeze(0))
output_function_test_4 = fno(input_function_train[4,:,:].unsqueeze(0))
output_function_test_5 = fno(data_read[:,175:210].to(device))

saved_pred[:,:,0] /= max_Tf
saved_pred[:,:,1] /= max_Ts
print(saved_pred.shape)
input_function_pred = 602168.58 * torch.cat((input_function_train[1,:,2], input_function_train[2,:,2], input_function_train[3,:,2], input_function_train[4,:,2], data_read[:,175:210,2].squeeze(0).to(device), test_read), dim=0)
output_function_pred = torch.cat((output_function_test_0, output_function_test_1, output_function_test_2, output_function_test_3, output_function_test_4, saved_pred), dim=1)
print(input_function_pred.shape, output_function_pred.shape)

input_function_plot = 602168.58 * torch.cat((input_function_train[0,:,2], input_function_train[1,:,2], input_function_train[2,:,2], input_function_train[3,:,2], input_function_train[4,:,2], data_read[:,175:210,2].squeeze(0).to(device)), dim=0)
output_function_plot = torch.cat((input_function_train[0,:,0:2], input_function_train[1,:,0:2], input_function_train[2,:,0:2], input_function_train[3,:,0:2], input_function_train[4,:,0:2], output_function_train[4,:,0:2]), dim=0)



plt.figure(dpi=1000)
plt.grid(True, which="both", ls=":")
print(output_function_pred.shape)
plt.plot(input_function_pred.cpu().detach(), max_Tf * output_function_pred[:,:,0].squeeze(0).cpu().detach(), label="Approximate Solution", c="C1", lw=0.5)
plt.plot(input_function_plot.cpu().detach(), max_Tf * output_function_plot[:,0].cpu().detach(), label="True Solution", c="C0", lw=0.5)


plt.legend()
plt.show()

plt.figure(dpi=250)
plt.grid(True, which="both", ls=":")

plt.plot(input_function_pred.cpu().detach(), max_Ts * output_function_pred[:,:,1].squeeze(0).cpu().detach(), label="Approximate Solution", c="C1", lw=2)
plt.plot(input_function_plot.cpu().detach(), max_Ts * output_function_plot[:,1].cpu().detach(), label="True Solution", c="C0", lw=2)


plt.legend()
plt.show()


In [ ]:
# Create a DataFrame from the numpy array
dt = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\TestingData.txt')
test_read = dt.iloc[:,0:1].values
test_read = torch.tensor(test_read, dtype = torch.float64).squeeze(1).to(device)
submission = torch.empty(34,3)
submission[:, 0] = test_read
submission[:, 1:3] = output_function_test_5[0,:34, 0:2]
submission[:, 1] *= max_Tf
submission[:, 2] *= max_Ts


submission = pd.DataFrame(submission.detach().numpy())

# Save the DataFrame as a text file
submission.to_csv(r'C:\Users\matth\OneDrive\Documents\tensor_data.txt', sep=',', index=False, header=['t', 'tf0', 'ts0'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
data_tens = pd.read_csv(r'C:\Users\matth\OneDrive\Documents\tensor_data.txt')
print(data_tens.shape)
subp = pd.read_csv(r'C:\Users\matth\Downloads\submissiontask3.txt')
y_1 = data_tens.iloc[:,0:3].values
y_2 = subp.iloc[:,0:3].values

y_1 = torch.tensor(y_1).to(torch.float64).squeeze(1)
y_2 = torch.tensor(y_2).to(torch.float64).squeeze(1)



plt.figure(dpi=250)
plt.grid(True, which="both", ls=":")

plt.plot(y_1[:,0].cpu().detach(), y_1[:,1].cpu().detach(), label="Matthias Solution", c="C1", lw=1)
plt.plot(y_2[:,0].cpu().detach(), y_2[:,1].cpu().detach(), label="Paul Solution", c="C0", lw=1)
plt.legend()
plt.show()
